In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import unidecode
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
import gensim.corpora as corpora
import gensim
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [3]:
df = pd.read_csv('DF-FINAL-GENSIM.csv')

In [5]:
df['titulo'].fillna('', inplace=True)
df['texto'].fillna('', inplace=True)

In [6]:
df['texto'].isna().value_counts()

False    3980
Name: text, dtype: int64

In [7]:
df['title_text'] = df['titulo'] + ' ' + df['texto']

In [4]:
df = df.dropna()

In [ ]:
df['title_text_ed']=df['title_text'].str.lower() # Pasa todo a minúsucula
df['title_text_ed']=df['title_text_ed'].str.replace(r'^https?:\/\/.*[\r\n]*', r'') # Elimina sitios web
df['title_text_ed']=df['title_text_ed'].str.replace(r'^http?:\/\/.*[\r\n]*', r'') # Elimina sitios web
df['title_text_ed']=df['title_text_ed'].str.replace(r'[¡|¿|?|!|\'|"|#|-|&|%|$|*]',r'') # ELimina puntuación
df['title_text_ed']=df['title_text_ed'].str.replace(r'[.|,|:|;|)|(|\|/]',r' ') # ELimina puntuación
df['title_text_ed']=df['title_text_ed'].str.replace(r'[0-9]+',r' ') # ELimina dígitos
df['title_text_ed']=df['title_text_ed'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [5]:
df['title_text_ed']=df['title_text_ed'].str.split()

In [12]:
stop=open('stopwordscustomnew.txt','r', encoding='utf-8').read().split('\n')
stop=[unidecode.unidecode(x) for x in stop]

In [13]:
df['title_text_ed']=df['title_text_ed'].apply(lambda x: ' '.join([item for item in x if item not in stop]))

In [14]:
df['title_text_ed']=df['title_text_ed'].str.split()

In [42]:
corpus = df['title_text_ed']

In [43]:
#TRAIN BIGRAMS
bigram = gensim.models.Phrases(corpus, min_count=5, threshold=6.5) # higher threshold fewer phrases.
bigram_mod = gensim.models.phrases.Phraser(bigram)
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

data_words_bigrams = make_bigrams(corpus)
id2word = corpora.Dictionary(data_words_bigrams)
#id2word.filter_extremes(no_below=3, no_above=0.95)
corpus = [" ".join(x) for x in data_words_bigrams]

In [44]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.96, 
                                   min_df=0.027,
                                   use_idf=True,
                                   norm='l1')

tfidf = tfidf_vectorizer.fit_transform(corpus)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [45]:
lda = LatentDirichletAllocation(n_components=10,
                                max_iter=100, 
                                learning_method='online',
                                random_state=255)
corpus_topics = lda.fit_transform(tfidf)

In [46]:
pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vectorizer,mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.009593  0.101115       1        1  15.043411
9      0.143024  0.141361       2        1  14.049950
0      0.111193 -0.021262       3        1  12.310672
2      0.030457  0.042066       4        1  10.571596
1      0.013881 -0.061388       5        1  10.309705
6      0.122488  0.045746       6        1   9.790145
4      0.024195  0.215557       7        1   9.314048
3      0.174241 -0.100625       8        1   7.803891
5     -0.386609  0.074637       9        1   5.598707
7     -0.242464 -0.437207      10        1   5.207876, topic_info=            Term         Freq        Total Category  logprob  loglift
256       juicio  3802.000000  3802.000000  Default  30.0000  30.0000
122      defensa  3695.000000  3695.000000  Default  29.0000  29.0000
65   casa_rosada  3729.000000  3729.000000  Default  28.0000  28.0000
113       debate  2152.000000  2152.000000  Default  27.0000  27.0000
34          auto  2029.000000  2029.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
516        viene     1.210272   640.901083  Topic10  -9.7580  -3.3170
140      dificil     1.209389   498.595444  Topic10  -9.7588  -3.0667
401     problema     1.209210   627.414118  Topic10  -9.7589  -3.2966
351     palabras     1.209101   513.724703  Topic10  -9.7590  -3.0968
31      asimismo     1.208931   564.943551  Topic10  -9.7591  -3.1920

[444 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.056875  abogado
0         3  0.035355  abogado
0         4  0.052264  abogado
0         5  0.195220  abogado
0         6  0.021520  abogado
...     ...       ...      ...
529       6  0.236340     zona
529       7  0.030394     zona
529       8  0.014673     zona
529       9  0.393027     zona
529      10  0.000524     zona

[2777 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 10, 1, 3, 2, 7, 5, 4, 6, 8])

In [47]:
p = pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vectorizer,mds='mmds')
pyLDAvis.save_html(p, 'final_bigrams.html')

In [48]:
corpus_topics=pd.DataFrame(corpus_topics)

In [49]:
corpus_topics.columns = ["Topic_" + str(i) for i in np.arange(1,11)]

In [50]:
df_topics = pd.concat([df,corpus_topics], axis=1)

In [51]:
df_topics.to_csv('comentarios_topicos_bigramas.csv',index=False)

In [52]:
df_topics.head()

,titulo,texto,DATE,DocumentIdentifier,V2Themes,title_text,title_text_ed,Medio,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9,Topic_10
0,"Tinelli, a punto de venderle sus acciones a López",De dueño a empleado. Ese el recorrido que hará...,2016-04-10,http://www.clarin.com/extrashow/tv/Tinelli-pun...,"ECON_STOCKMARKET,297;EPU_ECONOMY_HISTORIC,2935...","Tinelli, a punto de venderle sus acciones a Ló...","[tinelli, punto, venderle, acciones, lopez, du...",Clarín,0.058963,0.052494,0.052796,0.052168,0.525226,0.050009,0.053053,0.050000,0.053283,0.052008
1,Los bonos de la emisión no tendrán calificació...,El road show para salir a buscar al mercado un...,2016-04-10,http://www.ieco.clarin.com/bonos-emision-calif...,"BLOCKADE,2463;SEIGE,2463;TAX_FNCACT_DEPUTY,120...",Los bonos de la emisión no tendrán calificació...,"[bonos, emision, calificacion, default, road, ...",Clarín,0.050016,0.050010,0.050023,0.050037,0.050025,0.050004,0.050015,0.050030,0.050020,0.549820
2,"Solteras a los 27, el drama de las ""mujeres so...",Las mujeres solteras mayores de 27 años en Chi...,2016-04-10,http://www.lanacion.com.ar/1887974-solteras-a-...,"GENERAL_GOVERNMENT,921;GENERAL_GOVERNMENT,1105...","Solteras a los 27, el drama de las ""mujeres so...","[solteras, drama, mujeres, sobrantes, china, m...",La Nación,0.050039,0.549673,0.050045,0.050034,0.050020,0.050013,0.050021,0.050006,0.050031,0.050118
3,Santa Cruz: Gendarmería custodia una escribaní...,Una patrulla de Gendarmería Nacional custodia ...,2016-04-10,http://www.telam.com.ar/notas/201604/142737-la...,"DELAY,1040;USPEC_UNCERTAINTY1,1040;WB_2468_CON...",Santa Cruz: Gendarmería custodia una escribaní...,"[santa, cruz, gendarmeria, custodia, escribani...",Télam,0.050066,0.050069,0.050051,0.050057,0.424447,0.050024,0.175210,0.050000,0.050034,0.050041
4,El video citado por Capriles donde Hugo Chávez...,Manifestantes quemaron una estatua de Hugo Chá...,2016-04-10,http://www.clarin.com/mundo/Capriles-Hugo-Chav...,"SANCTIONS,257;DISABILITY,399;",El video citado por Capriles donde Hugo Chávez...,"[video, citado, capriles, chavez, apoya, refer...",Clarín,0.050034,0.050009,0.050005,0.549929,0.050005,0.050000,0.050011,0.050000,0.050003,0.050004


In [53]:
n_top_words = 40
topic_list=[]
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
       # message = "Topic #%d: " % topic_idx
        message = " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        topic_list.append(message)
    
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [54]:

print_top_words(lda, tfidf_feature_names, n_top_words)

In [55]:
topic_list

['vida mujeres mundo obra musica libro trabajo argentina gente forma publico espacio aires ciudad mujer casa idea tipo noche grupo hombre director cuerpo serie obras genero muestra nombre tema caso contar mail tierra estudio realidad experiencia chicos importante cultura proyecto',
 'mujer casa hijo familia madre joven hombre policia padre hija victima murio pareja padres conto hijos cuerpo muerte caso chicos vida fiscal esposa familiares ninos ocurrio escuela jovenes justicia ciudad noche relato zona auto menor detenido meses causa sociedad papa',
 'unidos gobierno region presidente paises paz argentina papa mundo europa acuerdo guerra derechos_humanos seguridad estadounidense anuncio comunicado politica ciudad ministro brasil pueblo apoyo mandatario internacional grupo visita organizacion encuentro semana ataque nacional reunion dialogo york violencia frente conflicto autoridades espana',
 'casa_rosada video redes_sociales fotos modelo pareja foto programa periodista imagenes imagen 

In [56]:
df_words_final=pd.DataFrame({"T1":[topic_list[0]],
                       "T2":[topic_list[1]],
                       "T3":[topic_list[2]],
                       "T4":[topic_list[3]],
                       "T5":[topic_list[4]],
                       "T6":[topic_list[5]],
                       "T7":[topic_list[6]],
                       "T8":[topic_list[7]],
                       "T9":[topic_list[8]],
                       "T10":[topic_list[9]]
                            })

In [57]:
df_words_final

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10
0,vida mujeres mundo obra musica libro trabajo a...,mujer casa hijo familia madre joven hombre pol...,unidos gobierno region presidente paises paz a...,casa_rosada video redes_sociales fotos modelo ...,fiscal causa juez justicia kirchner caso lopez...,policia auto sociedad plata hombre noche zona ...,partido equipo ciudad minutos rosario carrera ...,juicio defensa debate confirmo ciudadanos inve...,macri gobierno presidente scioli massa maurici...,argentina ciento millones gobierno economia pe...


In [58]:
df_words_final.to_csv('top_words_bigramas.csv', index=False)

In [ ]:
df_test = df_topics.sample(50000)

In [ ]:
t13 = df_test[df_test["Topic_13"] > 0.3]

In [ ]:
df_topics.info()